In [ ]:
#### import global modules
import os
import sys
import pandas as pd
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
pd.options.display.max_rows = 200

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Telus_GCP_Churn_Query= '''

select *
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
CROSS JOIN UNNEST(list_tpl_pi_alias_str) as param
WHERE DATE(prod_instnc_ts) = "2022-08-31" #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and pi_prod_instnc_stat_cd in ('C') #Canceled service
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and DATE(pi_prod_instnc_stat_ts)>='2022-08-01' and DATE(pi_prod_instnc_stat_ts)<'2022-09-01' #Date on which the status was updated


'''


In [ ]:
Telus_GCP_Churn_DF= extract_bq_data(bq_client,sql=Telus_GCP_Churn_Query)

In [ ]:
Telus_GCP_Churn_DF.info(verbose=True)

In [ ]:
Telus_GCP_Churn_DF.head()

In [ ]:
Telus_GCP_Churn_DF['pi_alias_str_key'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['pi_alias_str_value'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['bacct_bus_bacct_num'].head()

In [ ]:
Telus_GCP_Churn_DF['bacct_billg_acct_id'].head()

In [ ]:
Telus_GCP_Churn_DF['cust_bus_cust_id'].head()

In [ ]:
Telus_GCP_Churn_DF['cust_bus_cust_id'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['consldt_cust_bus_cust_id'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['bacct_bus_bacct_num'].value_counts()

In [ ]:
Telus_GCP_Churn_DF_cust=Telus_GCP_Churn_DF[Telus_GCP_Churn_DF['bacct_bus_bacct_num']==605276646]

In [ ]:
Telus_GCP_Churn_DF_cust

In [ ]:
Telus_GCP_Churn_DF_cust['pi_prod_instnc_stat_actvy_cd'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['pi_stat_actvy_rsn_cd'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['bacct_brand_nm'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['pi_prev_bus_bacct_num'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['pi_actvn_dlr_cd'].value_counts()

In [ ]:
Telus_GCP_Churn_DF['Telus_Churn']=1

In [ ]:
Telus_GCP_Churn_DF_sub=Telus_GCP_Churn_DF[['prod_instnc_ts','cust_bus_cust_id','bacct_bus_bacct_num','pi_prod_instnc_typ_cd','pi_prod_instnc_stat_ts','Telus_Churn']]

In [ ]:
Telus_GCP_Churn_DF_sub.info()

In [ ]:
Telus_GCP_Churn_DF_sub['pi_prod_instnc_stat_ts'].value_counts()

In [ ]:
Customer_Details_Query='''

SELECT 
customer_id,dealer_customer_id,primary_login_id,dealer_name,title,lastname,firstname,address1,address2,city,state,postal_code,join_date,account_type_name,customer_type_name,primary_email,primary_phone,last_updt_ts 
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where last_updt_ts= '2022-08-31 11:59:52.034161 UTC'
and account_type_name!='Standalone'
and customer_type_name='Customer'
order by dealer_name

'''

In [ ]:
ADC_Aug2022_CustomerBase= extract_bq_data(bq_client,sql=Customer_Details_Query)

In [ ]:
ADC_Aug2022_CustomerBase.info()

In [ ]:
ADC_Aug2022_CustomerBase.head()

In [ ]:
ADC_Aug2022_CustomerBase['dealer_name'].value_counts()

In [ ]:
sum(ADC_Aug2022_CustomerBase['dealer_customer_id'].isnull())

In [ ]:
ADC_Aug2022_CustomerBase['dealer_customer_id'].value_counts()

In [ ]:
ADC_Aug2022_CustomerBase_telus=ADC_Aug2022_CustomerBase[ADC_Aug2022_CustomerBase['dealer_name']=='TELUS Communications Inc.']

In [ ]:
ADC_Aug2022_CustomerBase_telus.info()

In [ ]:
ADC_Aug2022_CustomerBase_telus['customer_id'].value_counts()

In [ ]:
ADC_Aug2022_CustomerBase_telus['dealer_customer_id'].value_counts()

In [ ]:
ADC_Aug2022_CustomerBase_telus_merge=ADC_Aug2022_CustomerBase_telus.merge(Telus_GCP_Churn_DF_sub,how='left',left_on='dealer_customer_id',right_on='cust_bus_cust_id')

In [ ]:
ADC_Aug2022_CustomerBase_merge=ADC_Aug2022_CustomerBase.merge(Telus_GCP_Churn_DF_sub,how='left',left_on='dealer_customer_id',right_on='cust_bus_cust_id')

In [ ]:
ADC_Aug2022_CustomerBase_merge['Telus_Churn'].sum()

In [ ]:
ADC_Aug2022_CustomerBase_merge_1=ADC_Aug2022_CustomerBase_merge[ADC_Aug2022_CustomerBase_merge['Telus_Churn']==1]

In [ ]:
ADC_Aug2022_CustomerBase_merge_1['dealer_name'].value_counts()

In [ ]:
ADC_Aug2022_CustomerBase_merge_1.head()

In [ ]:
ADC_Aug2022_CustomerBase_merge_1['account_type_name'].value_counts()

In [ ]:
ADC_Aug2022_CustomerBase_telus_merge['Telus_Churn'].sum()

In [ ]:
ADC_Aug2022_CustomerBase_telus_merge['Telus_Churn'].isna().sum()/ADC_Aug2022_CustomerBase_telus_merge['Telus_Churn'].shape[0]

In [ ]:
Telus_GCP_Churn_DF_sub['Telus_Churn'].sum()

In [ ]:
ADC_Aug2022_CustomerBase_telus['ADC_flag']=1

In [ ]:
ADC_Aug2022_CustomerBase_telus.head()

In [ ]:
Telus_GCP_Churn_DF_sub_1=Telus_GCP_Churn_DF_sub.merge(ADC_Aug2022_CustomerBase_telus,how='inner',right_on='dealer_customer_id',left_on='cust_bus_cust_id')

In [ ]:
Telus_GCP_Churn_DF_sub_1.head()

In [ ]:
Telus_GCP_Churn_DF_sub_1.info()

In [ ]:
Telus_GCP_Churn_DF_sub_1['ADC_flag'].sum()